<a href="https://colab.research.google.com/github/BrownParticleAstro/dmtools_notebooks/blob/main/Demos/current_plot_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install matplotlib
!pip install plotly==5.23.0
!pip install -i https://test.pypi.org/simple/ dmtools-brown-edu --upgrade

Looking in indexes: https://test.pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 340.6 kB/s eta 0:00:00
  Attempting uninstall: dmtools-brown-edu
    Found existing installation: dmtools_brown_edu 0.0.15
    Uninstalling dmtools_brown_edu-0.0.15:
      Successfully uninstalled dmtools_brown_edu-0.0.15


In [5]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import io
import json
import os
import base64
from dotenv import load_dotenv
from typing import Dict, Any, Tuple, Callable, Optional
## only for local testing
from IPython.display import HTML, display

In [7]:
#wb_environment = 'jupyter'
wb_environment = 'colab'

In [8]:
def load_secrets(env_file_path=".env"):
    load_dotenv(env_file_path)
    email = os.getenv('EMAIL')
    api_key = os.getenv('API_KEY')
    if not email or not api_key:
        missing = []
        if not email:
            missing.append('EMAIL')
        if not api_key:
            missing.append('API_KEY')
        raise KeyError(f"Missing required secrets: {', '.join(missing)}")
    return {"email": email, "api_key": api_key}
cwd = os.getcwd()
print("Current working directory:", cwd)
if wb_environment == 'jupyter':
    secrets = load_secrets("./secrets/.env")
    email = secrets["email"]
    api_key = secrets["api_key"]
    print(email)

Current working directory: /content


In [9]:
if wb_environment == 'colab':
    from google.colab import userdata
    api_key = userdata.get('DMTOOLS_APIKEY')
    email = userdata.get("DMTOOLS_EMAIL")


In [10]:
from dmtools_brown_edu.dmtools_client_package.dmtools_quart import APIClient

In [11]:
Client = APIClient(email, api_key, suppress_logs=False, internal=False)

In [12]:
def get_empty_plot():
    empty_plot = {
      "dmtools_current_plot": {
        "plot_node": {
          "record": {
            "id": -1,
            "type": "Plot",
            "created": "2025-07-26T14:46:37.430Z",
            "updated": "2025-07-26T14:46:37.430Z"
          },
          "properties": {
            "name": "Default Plot",
            "xMax": "100",
            "xMin": "0",
            "yMax": "100",
            "yMin": "0",
            "xUnits": "keV/c^2",
            "yUnits": "cm^2",
            "plotType": "Cross Section vs WIMP Mass"
          }
        },
        "display_data": [
          {
            "data": [
              {
                "record": {
                  "id": -1,
                  "type": "Data",
                  "created": "2025-07-26T14:46:37.430Z",
                  "updated": "2025-07-26T14:46:37.430Z"
                },
                "properties": {
                  "raw": "[[['0','0'],['0','1'],['1','1'],['1','0'],['0','0']]]",
                  "open": 0,
                  "year": 2000,
                  "label": "label",
                  "hepUrl": "https://www.hepdata.net/record/show/hepdata.123456",
                  "public": 0,
                  "rating": 0,
                  "values": [
                    [
                      ["0", "0"],
                      ["0", "1"],
                      ["1", "1"],
                      ["1", "0"],
                      ["0", "0"]
                    ]
                  ],
                  "xUnits": "GeV/c^2",
                  "yUnits": "cm2",
                  "comment": "comment",
                  "dateEnd": None,
                  "official": 0,
                  "xRescale": "1",
                  "yRescale": "1",
                  "dateStart": None,
                  "reference": "reference",
                  "experiment": "Experiment",
                  "resultType": "Th",
                  "greatestHit": 0,
                  "label_short": "label",
                  "dateOfficial": None,
                  "defaultColor": "black",
                  "defaultStyle": "line",
                  "spinDependency": "SD",
                  "measurementType": "Unknown",
                  "dateAnnouncement": None
                }
              }
            ],
            "record": {
              "id": -1,
              "type": "Display",
              "created": "2025-07-26T14:46:37.430Z",
              "updated": "2025-07-26T14:46:37.430Z"
            },
            "properties": {
              "color": "black",
              "style": "line"
            }
          }
        ]
      }
    }
    return empty_plot

In [13]:
def get_color_mpl(color_in):
    ## converts dmtools color into matplotlib color
    trace_color = {}

    # Define the base color and alpha based on the input
    if color_in in ('k', 'black', 'Blk'):
        trace_color.update({'color': 'black', 'alpha': 1})
    elif color_in in ('r', 'red', 'Red', 'dkr'):
        trace_color.update({'color': 'red', 'alpha': 1})
    elif color_in in ('dkg', 'DkG', 'green', 'Grn'):
        trace_color.update({'color': 'green', 'alpha': 1})
    elif color_in in ('ltg', 'LtG'):
        trace_color.update({'color': 'green', 'alpha': 0.5})
    elif color_in in ('ltr', 'LtR'):
        trace_color.update({'color': 'red', 'alpha': 0.5})
    elif color_in == 'b':
        trace_color.update({'color': 'blue', 'alpha': 1})
    elif color_in in ('ltb', 'LtB', 'Blue','Blu','DkB'):
        trace_color.update({'color': 'blue', 'alpha': 0.5})
    elif color_in in ('c', 'Cyan', 'cyan'):
        trace_color.update({'color': 'cyan', 'alpha': 1})
    elif color_in in ('g', 'grey'):
        trace_color.update({'color': 'grey', 'alpha': 1})
    elif color_in in ('g10', 'g20', 'g30', 'g40', 'g50', 'g60', 'g70', 'g80', 'g90', 'G60'):
        trace_color.update({'color': 'grey'})
        try:
            shade = int(color_in[1:]) / 100
            trace_color.update({'alpha': shade})
        except:
            trace_color.update({'alpha': 1})
    elif color_in in ('m', 'magenta', 'Mag'):
        trace_color.update({'color': 'magenta', 'alpha': 1})
    elif color_in in ('y', 'yellow','Yel'):
        trace_color.update({'color': 'yellow', 'alpha': 1})
    elif color_in in ('w', 'white'):
        trace_color.update({'color': 'white', 'alpha': 1})
    else:
        trace_color.update({'color': 'black', 'alpha': 1})

    return trace_color


def get_style_mpl(color_in, style_in):
    # converts dmtools color style combination into matplotlib plot config
    trace_color = get_color_mpl(color_in)
    trace_style = trace_color.copy()  # Start by copying the color attributes

    # Set the style based on the style_in input
    if style_in in ('dot', 'dotted', 'Dot'):
        trace_style.update({
            'linestyle': ':',
            'linewidth': 1,
            'marker': None,
            'markersize': 0,
            'alpha': 1,
            'fill': False,
            'style': 'dot'
        })
    elif style_in in ('dash', 'Dash'):
        trace_style.update({
            'linestyle': '--',
            'linewidth': 1,
            'marker': None,
            'markersize': 0,
            'alpha': 1,
            'fill': False,
            'style': 'dash'
        })
    elif style_in in ('fill', 'Fill'):
        trace_style.update({
            'linestyle': None,
            'linewidth': 0,
            'marker': None,
            'markersize': 0,
            'alpha': 0.3,
            'fill': True,
            'style': 'fill'
        })
    elif style_in in ('Line', 'line', 'lines'):
        trace_style.update({
            'linestyle': '-',
            'linewidth': 1,
            'marker': None,
            'markersize': 0,
            'alpha': 1,
            'fill': False,
            'style': 'line'
        })
    elif style_in == "point":
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': '.',
            'markersize': 8,
            'alpha': 1,
            'fill': False,
            'style': 'point'
        })
    elif style_in in ('cross', 'Cross'):
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': 'x',
            'markersize': 8,
            'alpha': 1,
            'fill': False,
            'style': 'cross'
        })
    elif style_in == 'circle':
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': 'o',
            'markersize': 8,
            'alpha': 1,
            'fill': False,
            'style': 'circle'
        })
    elif style_in == 'plus':
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': '+',
            'markersize': 8,
            'alpha': 1,
            'fill': False,
            'style': 'plus'
        })
    elif style_in in ('asterisk', 'star'):
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': '*',
            'markersize': 12,
            'alpha': 1,
            'fill': False,
            'style': 'star'
        })
    elif style_in in ('pentagon', "pent"):
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': "p",
            'markersize': 10,
            'alpha': 1,
            'fill': False,
            'style': 'pentagon'
        })
    elif style_in in ('hex', 'hexagon'):
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': 'h',
            'markersize': 10,
            'alpha': 1,
            'fill': False,
            'style': 'hexagon'
        })
    elif style_in in ('triu', 'triangle-up'):
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': "^",
            'markersize': 10,
            'alpha': 1,
            'fill': False,
            'style': 'triangle-up'
        })
    elif style_in in ('trid', 'triangle-down'):
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': "v",
            'markersize': 10,
            'alpha': 1,
            'fill': False,
            'style': 'triangle-down'
        })
    elif style_in in ('tril','triangle-left') :
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': "<",
            'markersize': 10,
            'alpha': 1,
            'fill': False,
            'style': 'triangle-left'
        })
    elif style_in in ('trir', 'triangle-right') :
        trace_style.update({
            'linestyle': 'None',
            'linewidth': 1,
            'marker': ">",
            'markersize': 10,
            'alpha': 1,
            'fill': False,
            'style': 'triangle-right'
        })
    else:
        trace_style.update({
            'linestyle': '-',
            'linewidth': 1,
            'marker': None,
            'markersize': 0,
            'alpha': 1,
            'fill': False,
            'style': 'line'
        })

    return trace_style

def get_clean_color_style(color_in, style_in):
    # this cleans old dmtools styles into a consistent style

    clean_trace_dict = get_color_mpl(color_in)
    clean_trace_color = clean_trace_dict['color']

    # Set the style based on the style_in input
    if style_in in ('dot', 'dotted', 'Dot'):
        clean_trace_style = 'dotted'
    elif style_in in ('dash', 'Dash'):
        clean_trace_style = 'dash'
    elif style_in in ('fill', 'Fill'):
        clean_trace_style = 'fill'
    elif style_in in ('Line', 'line', 'lines'):
        clean_trace_style = 'line'
    elif style_in == 'point':
        clean_trace_style = 'point'
    elif style_in in ('cross', 'Cross'):
        clean_trace_style = 'cross'
    elif style_in == 'circle':
        clean_trace_style = 'circle'
    elif style_in == 'plus':
        clean_trace_style = 'cross'
    elif style_in in ('asterisk', 'star'):
        clean_trace_style = 'star'
    elif style_in in ('pentagon', 'pent'):
        clean_trace_style = 'pentagon'
    elif style_in in ('hex', 'hexagon'):
        clean_trace_style = 'hexagon'
    elif style_in in ('triu', 'triangle', 'triangle-up'):
        clean_trace_style = 'triangle-up'
    elif style_in == ('trid', 'triangle', 'triangle-down'):
        clean_trace_style = 'triangle-down'
    elif style_in == ('tril','triangle-left') :
        clean_trace_style = 'triangle-left'
    elif style_in == ('trir', 'triangle-right') :
        clean_trace_style = 'triangle-right'
    else:
        clean_trace_style = 'line'

    return clean_trace_color, clean_trace_style

In [14]:
## Unit scaling
## Need help when creating brand new plot and what the default scale should be?
unit_factors = {
    'eV': 1e0,
    'keV': 1e3,
    'MeV': 1e6,
    'GeV': 1e9,
    'TeV': 1e12
}

def normalize_unit(unit):
    # Remove '/c^2', '/c2', '/c²', etc. and whitespace
    #print("unit >>", unit)
    try:
        unit = unit.strip()
        if '/c' in unit:
            unit = unit.split('/c')[0]
    except:
        unit = unit

    return unit

def convert_mass_units(value, from_unit, to_unit):
    """
    Convert a mass value (or array) from one energy/c^2 unit to another.
    from_unit and to_unit can be like 'GeV', 'GeV/c^2', 'MeV/c2', etc.
    """
    from_unit_norm = normalize_unit(from_unit)
    to_unit_norm = normalize_unit(to_unit)
    #print("convert_mass_units : from ", from_unit_norm, " to ", to_unit_norm)
    if from_unit_norm not in unit_factors or to_unit_norm not in unit_factors:
        raise ValueError(f"Supported units: {list(unit_factors.keys())}")
    try:
        value_eV = value * unit_factors[from_unit_norm]
        result = value_eV / unit_factors[to_unit_norm]
    except:
        result = 0
    return result

allowed_units = ['eV', 'keV', 'MeV', 'GeV', 'TeV']

def get_x_label(selected_unit):
    selected_unit = normalize_unit(selected_unit)
    if selected_unit not in allowed_units:
        #raise ValueError(f"Unit must be one of: {allowed_units}")
        selected_unit = 'GeV'  # Default to GeV if invalid unit is provided
    return r"$\mathrm{WIMP\ Mass}\ [\mathrm{" + selected_unit + r"}/c^{2}]$"

In [15]:
r_51 = Client.read(id=51)
r_51
# r_51 is a list of lists = read can be used to retrieve one or many nodes
# a node retrieved is always its database record and its json properties as a list
# the following shows the data with id of 51 being retrieved and then split into record and properties
#r_51_record = r_51[0][0]
#r_51_properties = r_51[0][1]

#r_52 = Client.read(id=52)
#r_52_record = r_52[0][0]
#r_52_properties = r_52[0][1]
#r_52_properties

2025-08-06 13:26:11,278 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


[{'properties': {'comment': 'Sigma normalized to WIMP-Nucleon, no background subtraction',
   'dateAnnouncement': '1999-01-01',
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'blue',
   'defaultStyle': 'dash',
   'experiment': 'CDMS I (SUF)',
   'greatestHit': 0,
   'label': 'CDMS 1999 4 kg-day Ge',
   'label_short': 'CDMS 1999 4 kg-day Ge',
   'measurementType': 'Dir',
   'official': 1,
   'open': 0,
   'public': 0,
   'rating': 90,
   'raw': '{[6.3095734 1.8632413e-37;8.1347616 2.6429088e-39;10.487927 1.807643e-40;13.521799 3.3370489e-41;17.433288 1.2176643e-41;22.476265 7.1061293e-42;28.978037 5.7609944e-42;37.360596 5.6309457e-42;48.168001 6.1214702e-42;62.101694 7.0796666e-42;80.066026 8.5454071e-42;103.22695 1.0551458e-41;133.0877 1.3144547e-41;171.58636 1.6630495e-41;221.22163 2.1110566e-41;285.21503 2.6923299e-41;367.71999 3.4462025e-41;474.09139 4.4171161e-41;611.23315 5.6721956e-41;788.04628 7.2906839e-41;1016.0067 9.3732685e-41;1309.909

In [16]:
# Query Examples
# NB Client Read ALWAYS returns a list of Dictionaries so to access any record you need to choose it by index [0]
# Any occurrence of CDMS and match phrase "Sigma normalized"
r_1 = Client.read('CDMS "Sigma normalized"')
# Any occurrence of CDMS and not in 1999
r_2 = Client.read('CDMS -1999')
# Data record with id=52
#r_3 = Client.read('id:52')
#or
r_3 = Client.read(id=52)
# Data records for experiment 'CDMS I (SUF)'
r_4 = Client.read('experiment:CDMS I (SUF)')
len(r_3)
r_3

2025-08-06 13:26:21,394 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API
2025-08-06 13:26:22,785 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API
2025-08-06 13:26:25,310 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API
2025-08-06 13:26:25,954 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


[{'properties': {'comment': 'Sigma normalized to WIMP-Nucleon, Background subtracted',
   'dateAnnouncement': '1999-01-01',
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'blue',
   'defaultStyle': 'line',
   'experiment': 'CDMS I (SUF)',
   'greatestHit': 0,
   'label': 'CDMS 1999 4 kg-day Ge with neutron bg subtract.',
   'label_short': 'CDMS 1999 4 kg-day Ge with neutron bg subtract.',
   'measurementType': 'Dir',
   'official': 1,
   'open': 0,
   'public': 0,
   'rating': 80,
   'raw': '{[6.3095734 1.8228551e-37;8.1347616 2.5855165e-39;10.487927 1.7657822e-40;13.521799 3.2271517e-41;17.433288 1.1331211e-41;22.476265 6.2303537e-42;28.978037 4.7098555e-42;37.360596 4.3225733e-42;48.168001 4.53597e-42;62.101694 5.2108423e-42;80.066026 6.1999942e-42;103.22695 7.8027863e-42;133.0877 9.5350325e-42;171.58636 1.2032764e-41;221.22163 1.5279808e-41;285.21503 1.951882e-41;367.71999 2.4957172e-41;474.09139 3.2035739e-41;611.23315 4.1142131e-41;788.04628 

In [17]:
## Read always retrieves a list of dicts
r_51 = Client.read(id=51)
r_51_len = len(r_51)
print("len of r51 >>", r_51_len)
first_data = r_51_len - 1 ## python lists start at 0
r_51_record = r_51[first_data]['record']
r_51_properties = r_51[first_data]['properties']
r_52 = Client.read(id=52)
r_52_record = r_52[first_data]['record']
r_52_properties = r_52[first_data]['properties']

2025-08-06 13:26:37,119 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API
2025-08-06 13:26:37,764 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API
INFO:dmtools_brown_edu.dmtools_client_package.dmtools_quart:Calling read data API


len of r51 >> 1


In [ ]:
#r_51

In [ ]:
'''
def get_empty_plot_data():
    import json

    # Open the file and load its contents into a Python variable
    with open('./data/empty_plot.json', 'r') as f:
        json_data = json.load(f)

    return json_data
'''

def get_current_plots_data():
    mode = 'api'
    if mode == 'json':
        # Print the current working directory
        print("Current working directory:", os.getcwd())

        # Open the file and load its contents into a Python variable
        with open('./data/current_plot_diagnostics.json', 'r') as f:
            json_data = json.load(f)
    elif mode == 'api':
        r_51 = Client.read(id=51)
        r_51_len = len(r_51)
        first_data = 0
        r_51_record = r_51[first_data]['record']
        r_51_properties = r_51[first_data]['properties']
        r_52 = Client.read(id=52)
        r_52_record = r_52[first_data]['record']
        r_52_properties = r_52[first_data]['properties']

        plot_framework = get_empty_plot()
        plot_framework['dmtools_current_plot']['plot_node']['properties']['xUnits'] = 'GeV/c^2'
        plot_framework['dmtools_current_plot']['plot_node']['properties']['yMax'] = -26
        plot_framework['dmtools_current_plot']['plot_node']['properties']['yMin'] = -54
        plot_framework['dmtools_current_plot']['plot_node']['properties']['xMax'] = 10000
        plot_framework['dmtools_current_plot']['plot_node']['properties']['xMin'] = 1
        plot_framework['dmtools_current_plot']['plot_node']['properties']['name'] = "Plot 2025 08 05 1148"


        plot_framework['dmtools_current_plot']['display_data'][0]['data'][0]['record'] = r_51_record
        plot_framework['dmtools_current_plot']['display_data'][0]['data'][0]['properties'] = r_51_properties

        plot_framework['dmtools_current_plot']['display_data'].append({'data': [{}]})
        plot_framework['dmtools_current_plot']['display_data'][1]['data'].append({})

        plot_framework['dmtools_current_plot']['display_data'][1]['data'][0]['record'] = r_52_record
        plot_framework['dmtools_current_plot']['display_data'][1]['data'][0]['properties'] = r_52_properties

        json_data = plot_framework

    else:
        json_data = get_empty_plot_data()

    return json_data



In [ ]:
plot_data = get_current_plots_data()
plot_data

In [ ]:
#plot_framework = get_empty_plot_data()
#data_record = plot_framework['dmtools_current_plot']['display_data'][0]['data'][0]['data_record']
#data_properties = plot_framework['dmtools_current_plot']['display_data'][0]['data'][0]['data_properties']
#plot_framework['dmtools_current_plot']['display_data']
#data_properties

In [ ]:
#plot_framework['dmtools_current_plot']['display_data'][0]['data'][0]['data_record'] = r_51_record
#plot_framework['dmtools_current_plot']['display_data'][0]['data'][0]['data_properties'] = r_51_properties

#plot_framework['dmtools_current_plot']['display_data'].append({'data': [{}]})
#plot_framework['dmtools_current_plot']['display_data'][1]['data'].append({})

#plot_framework['dmtools_current_plot']['display_data'][1]['data'][0]['data_record'] = r_52_record
#plot_framework['dmtools_current_plot']['display_data'][1]['data'][0]['data_properties'] = r_52_properties

In [ ]:
def create_standard_figure():
    """Create a standard figure with consistent styling"""
    fig = plt.figure(figsize=(10, 10), linewidth=0, edgecolor='#D0D6DB', facecolor='#D0D6DB')
    gs = gridspec.GridSpec(64, 64)
    ax = fig.add_subplot(gs[2:63, 2:63])
    ax.set_facecolor('white')
    ax.tick_params(axis='both', labelsize=16)
    return fig, ax

def save_figure_to_base64(fig, format_in='svg', dpi=100):
    """Save a matplotlib figure to base64 encoded string"""
    img = io.BytesIO()
    fig.savefig(img, format=format_in, dpi=dpi, bbox_inches='tight', pad_inches=0.25)
    img.seek(0)
    plt.close(fig)  # Close the figure to free memory
    return base64.b64encode(img.getvalue()).decode('utf8')

def set_plot_data(conn='', plot_id=-1, data_id=-1, schema="data", function_mode="local", format_in='svg'):
    """
    Main function that delegates to specific plot functions based on plot type
    """
    # Get the data - either from local cache or from a connection
    if function_mode == "local":
        data = get_current_plots_data()
        plot_data = data.get('dmtools_current_plot')
        #print("plot_data>>>>>>>>>", plot_data)
    else:
        try:
            # This would be your remote data retrieval code
            plot_data = {}  # Placeholder
        except:
            plot_data = {}

    # Extract plot type and other key properties
    plot_node = plot_data.get("plot_node", {})
    plot_properties = plot_node.get("properties", {})
    plot_record = plot_node.get("record", {})
    plot_id = plot_record.get('id')
    plot_type = plot_properties.get("plotType", "Cross Section vs WIMP Mass")
    plot_name = plot_properties.get("name", f"Plot {plot_id}")

    # Delegate to the appropriate plot function based on plot type
    if plot_type == "Cross Section vs WIMP Mass":
        return plot_cross_section_vs_mass(plot_data, plot_id, plot_name, format_in=format_in)
    elif plot_type == "Cross Section / Mass [in GeV] vs Mass[GeV]":
        return plot_cross_section_div_mass(plot_data, plot_id, plot_name, format_in=format_in)
    elif plot_type == "Exclusion Curve":
        return plot_exclusion_curve(plot_data, plot_id, plot_name, format_in=format_in)
    else:
        # Default plot or unknown type
        return plot_default(plot_data, plot_id, plot_name)

def plot_cross_section_vs_mass(plot_data, plot_id, plot_name, format_in='svg'):
    """Create Cross Section vs WIMP Mass plot"""
    print('plotting - Cross Section vs WIMP Mass plot')
    fig, ax = create_standard_figure()

    # Extract properties
    plot_node = plot_data.get("plot_node", {})
    plot_properties = plot_node.get("properties", {})

    # Plot each dataset
    for display in plot_data.get("display_data", [{}]):
        display_properties = display.get('properties', {})
        style = display_properties.get('style', 'line')
        color = display_properties.get('color', 'black')

        for data in display.get("data", []):
            data_properties = data.get('properties', {})
            y_rescale = float(data_properties.get('yRescale', 1))
            x_rescale = float(data_properties.get('xRescale', 1))
            x_units = data_properties.get('xUnits', 1)

            list_data = data_properties.get('values', [[[0.0, 0.0], [0.0, 1.0]]])
            #print("list_data>>>>>" ,list_data)

            # Get style configuration
            line_kwargs = get_style_mpl(color, style)
            line_kwargs.pop('style', None)
            line_kwargs.pop('fill', None)

            fill_kwargs = get_style_mpl(color, style)
            fill_kwargs.pop('style', None)
            fill_kwargs.pop('marker', None)
            fill_kwargs.pop('markersize', None)
            fill_kwargs.pop('linestyle', None)
            fill_kwargs.pop('linewidth', None)
            fill_kwargs.pop('fill', None)

            print("fill_kwargs>>", fill_kwargs)

            #print("list_data >>>>>>>", list_data)

            # Plot each trace
            for trace in list_data:
                try:
                    x = [float(item[0]) * x_rescale for item in trace]
                    selected_unit = plot_properties.get('xUnits', 'GeV/c^2') ## will people know when to modify this??
                    unit_x = [convert_mass_units(val, x_units, selected_unit) for val in x]
                    y = [float(item[1]) * y_rescale for item in trace]

                    ax.plot(unit_x, y, **line_kwargs)
                    if style == 'fill':
                        ax.fill_between(unit_x, y, **fill_kwargs)
                except:
                    x = 1

    # Configure axes
    ax.set_xscale('log')
    ax.set_yscale('log')

    # Set ranges
    set_axis_ranges(ax, plot_properties)

    # Set labels
    ax.set_ylabel(r"$\mathrm{Cross\ Section}\ [cm^{2}]\ (\mathrm{normalized\ to\ nucleon})$", fontsize=18)
    selected_unit = plot_properties.get('xUnits', 'GeV/c^2') ## will people know when to modify this??
    ax.set_xlabel(get_x_label(selected_unit), fontsize=18)

    # Set title
    plot_title = f"{plot_name} ({plot_id})"
    ax.set_title(plot_title, fontsize=18)

    # Set log formatting
    set_log_formatting(ax)

    # Save and return
    encoded_img = save_figure_to_base64(fig, format_in=format_in)
    return plot_name, encoded_img

def plot_cross_section_div_mass(plot_data, plot_id, plot_name, format_in='svg'):
    """Create Cross Section / Mass vs Mass plot"""
    fig, ax = create_standard_figure()

    # Similar implementation to plot_cross_section_vs_mass but with
    # appropriate calculations for this plot type

    # Extract properties
    plot_node = plot_data.get("plot_node", {})
    plot_properties = plot_node.get("properties", {})

    # Plot each dataset
    for display in plot_data.get("display_data", [{}]):
        display_properties = display.get('properties', {})
        style = display_properties.get('style', 'line')
        color = display_properties.get('color', 'black')

        for data in display.get("data", []):
            data_properties = data.get('properties', {})
            y_rescale = float(data_properties.get('yRescale', 1))
            x_rescale = float(data_properties.get('xRescale', 1))
            x_units = data_properties.get('xUnits', 1)

            list_data = data_properties.get('values', [[[0.0, 0.0], [0.0, 1.0]]])

            # Get style configuration
            line_kwargs = get_style_mpl(color, style)
            line_kwargs.pop('style', None)
            line_kwargs.pop('fill', None)

            fill_kwargs = get_style_mpl(color, style)
            fill_kwargs.pop('style', None)
            fill_kwargs.pop('marker', None)
            fill_kwargs.pop('markersize', None)
            fill_kwargs.pop('linestyle', None)
            fill_kwargs.pop('linewidth', None)
            fill_kwargs.pop('fill', None)

            # Plot each trace - note the different calculation for y values
            for trace in list_data:
                x = [float(item[0]) * x_rescale for item in trace]
                unit_x = [convert_mass_units(val, x_units, 'GeV') for val in x]
                y = [(float(item[1]) * y_rescale) / xi for item, xi in zip(trace, unit_x)]

                ax.plot(unit_x, y, **line_kwargs)
                if style == 'fill':
                    #ax.fill_between(unit_x, y, **fill_kwargs)
                    ax.fill(unit_x, y, **fill_kwargs)

    # Configure axes and other settings like in the first function
    ax.set_xscale('log')
    ax.set_yscale('log')

    # Set ranges
    set_axis_ranges(ax, plot_properties)

    # Set labels with appropriate titles for this plot type
    ax.set_ylabel(r"$\mathrm{Cross\ Section/Mass}\ [cm^{2}/GeV]$", fontsize=18)
    ax.set_xlabel(r"$\mathrm{WIMP\ Mass}\ [GeV]$", fontsize=18)

    # Set title
    plot_title = f"{plot_name} ({plot_id})"
    ax.set_title(plot_title, fontsize=18)

    # Set log formatting
    set_log_formatting(ax)

    # Save and return
    encoded_img = save_figure_to_base64(fig, format_in=format_in)
    return plot_name, encoded_img

def plot_exclusion_curve(plot_data, plot_id, plot_name, format_in='svg'):
    """Create exclusion curve plot"""
    # Implementation for exclusion curves
    # Similar to previous functions but with specialized logic for exclusion curves
    fig, ax = create_standard_figure()

    # Implementation would go here...

    # Save and return
    encoded_img = save_figure_to_base64(fig)
    return plot_name, encoded_img

def plot_default(plot_data, plot_id, plot_name):
    """Create a default plot when type is unknown"""
    fig, ax = create_standard_figure()

    plot_data = get_empty_plot_data()

    # Simple implementation for generic data
    for display in plot_data.get("display_data", [{}]):
        for data in display.get("data", []):
            data_properties = data.get('properties', {})
            list_data = data_properties.get('values', [[[0.0, 0.0], [0.0, 1.0]]])

            for trace in list_data:
                x = [float(item[0]) for item in trace]
                y = [float(item[1]) for item in trace]
                ax.plot(x, y)

    ax.set_title(f"{plot_name} ({plot_id})")

    # Save and return
    encoded_img = save_figure_to_base64(fig)
    return plot_name, encoded_img


def set_axis_ranges(ax, plot_properties):
    """Set the X and Y axis ranges based on plot properties"""
    # X range
    xmin = plot_properties.get('xMin', '0')
    xmin = int(xmin)
    if xmin < 1:
        xmin = 1
    xmin = float(xmin) if xmin else 0
    xmax = plot_properties.get('xMax', '3')
    xmax = float(xmax) if xmax else 10000
    ax.set_xlim([xmin, xmax])

    # Y range (in log scale)
    ymin_exp = plot_properties.get('yMin', '-42')
    ymin_exp = float(ymin_exp) if ymin_exp else -42
    ymax_exp = plot_properties.get('yMax', '-42')
    ymax_exp = float(ymax_exp) if ymax_exp else -42
    ax.set_ylim([10**ymin_exp, 10**ymax_exp])

def set_log_formatting(ax):
    """Apply log formatting to axes"""
    try:
        from matplotlib.ticker import LogFormatterMathtext
        ax.xaxis.set_major_formatter(LogFormatterMathtext())
        ax.yaxis.set_major_formatter(LogFormatterMathtext())
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontname('DejaVu Sans')
    except:
        pass  # Skip if formatter isn't available

def display_plot(url_in, filetype="svg"):
    """
    Displays an SVG or PDF from a base64-encoded string in Jupyter or HTML.

    Args:
        dmtools_plot_url (str): The base64-encoded image.
        filetype (str): "svg" or "pdf"
    """
    if filetype == "svg":
        html = f'<img src="data:image/svg+xml;base64,{url_in}" style="max-width:600px;"/>'
    elif filetype == "pdf":
        # Use <embed> or <iframe> for PDF display
        html = (
            f'<embed src="data:application/pdf;base64,{url_in}" '
            'type="application/pdf" width="600" height="600"/>'
        )
    else:
        raise ValueError("filetype must be 'svg' or 'pdf'")
    display(HTML(html))

def make_plot(format_in='svg'):
    plot_name, plot_url = set_plot_data(format_in=format_in)
    display_plot(plot_url, filetype=format_in)

In [ ]:
# Your base64 string
plot_name, plot_url = set_plot_data(format_in='svg')

from IPython.display import Image, display

try:
    base64.b64decode(plot_url)
    print("Base64 is valid")
except Exception as e:
    print("Base64 is invalid:", e)

In [ ]:
make_plot(format_in='svg')